In [1]:
%pwd
%cd ..
%pwd

/Users/brunostordeur/Docs/GitHub/End-to-End-ML-Project-with-MLflow


'/Users/brunostordeur/Docs/GitHub/End-to-End-ML-Project-with-MLflow'

In [2]:
import os
import pandas as pd

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [4]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
    
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )
        return model_trainer_config
        

In [6]:
import os 
from mlProject import logger
from sklearn.linear_model import ElasticNet
import pandas as pd
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):

        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        lr = ElasticNet(
            l1_ratio = self.config.l1_ratio,
            alpha= self.config.alpha,
            random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [10]:
try:
    config = ConfigurationManager()
    trainer_config = config.get_model_trainer_config()
    trainer = ModelTrainer(config=trainer_config)
    trainer.train()
except Exception as e:
    raise e

[2023-08-07 16:00:07,387: INFO: common: yaml file: <_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfuly]
[2023-08-07 16:00:07,390: INFO: common: yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfuly]
[2023-08-07 16:00:07,392: INFO: common: yaml file: <_io.TextIOWrapper name='schema.yaml' mode='r' encoding='UTF-8'> loaded successfuly]
[2023-08-07 16:00:07,393: INFO: common: created directory at: artifacts]
[2023-08-07 16:00:07,393: INFO: common: created directory at: artifacts/model_trainer]
